# 1. 국민청원 데이터 추출 후 DataFrame에 담기
국민청원 첫 페이지 하단의 청원 목록에서 번호, 분류, 제목, 참여인원을 추출하여 DataFrame에 담아주세요. 

### 데이터 추출하기
국민청원 첫 페이지 하단의 청원 목록에서 번호, 분류, 제목, 참여인원을 추출하기

In [128]:
# 필요한 라이브러리 가져오기
!pip install html5lib beautifulsoup4

from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
#html 받아오기
url = "https://www1.president.go.kr/petitions"
with request.urlopen(url) as f:
  html = f.read().decode('utf-8')

In [130]:
# DOM 구성하기
bs = BeautifulSoup(html, 'html5lib')
requests = bs.select('.wrap > .board > .b_list > .bl_body > ul > li > .bl_wrap')

#데이터프레임 생성
request_people = pd.DataFrame(columns=['번호','분류','제목','참여인원'])

# 번호, 분류, 제목, 참여인원을 추출해서 넣기
for request in requests:
  ## 왜 클래스를 #이 아닌 .으로 가져와야 될까?
  num = request.select_one('.bl_no').text.replace("번호 ", "")
  category = request.select_one('.bl_category').text.replace("분류 ", "")
  subject = request.select_one('.bl_subject a').text.replace("제목 ", "")
  ## 왜인지 agree만 안가져와진다. 일단 패스
  # agree = request.select_one('.bl_agree')
  
  # 데이터프레임에 넣어주기
  ## efficient하게 넣기?
  request_people = request_people.append({"번호":num, "분류":category, \
                         "제목":subject, "참여인원":0}, \
                        ignore_index=True)

request_people.head()

,번호,분류,제목,참여인원
0,244368,반려동물,"국내 파,양,지 시장의 확산을 막는 환경청 바꾸자",0
1,244367,기타,노회찬 의원 수사와 부검을 해야합니다. 화장을 막아 ...,0
2,244366,문화/예술/체육/언론,한겨레 신문은 폐간해야 한다!!,0
3,244365,기타,오래된빛때문에 고통받는서민들,0
4,244364,교통/건축/국토,신안산선 연내착공을 부탁드립니다,0


# 2. 여러 페이지 데이터 가져오기
국민청원 목록 첫 페이지 주소는 https://www1.president.go.kr/petitions?page=1 입니다. 두번째 페이지 주소는 https://www1.president.go.kr/petitions?page=2 입니다. 처음 10개 페이지에 순차적으로 방문하여 번호, 분류, 제목, 참여인원을 추출하고 그 결과를 DataFrame에 담아주세요.


In [0]:
# url에서 html을 받아오는 함수
def get_html(url):
  with request.urlopen(url) as f:
    html = f.read().decode('utf-8')


# html에서 원하는 태그를 받아오는 함수
def get_div(html):
  bs = BeautifulSoup(html, 'html5lib')
  return bs.select('.wrap > .board > .b_list > .bl_body > ul > li > .bl_wrap')

# beautifulsoup 객체를 받아 
# 데이터 추출 후 데이터프레임에 넣어주는 함수
def html_to_df(bs, df):
  for request in requests:
    ## 왜 클래스를 #이 아닌 .으로 가져와야 될까?
    num = request.select_one('.bl_no').text.replace("번호 ", "")
    category = request.select_one('.bl_category').text.replace("분류 ", "")
    subject = request.select_one('.bl_subject a').text.replace("제목 ", "")
    ## 왜인지 agree만 안가져와진다. 일단 패스
    # agree = request.select_one('.bl_agree')

    # 데이터프레임에 넣어주기
    request_people = request_people.append({"번호":num, "분류":category, \
                           "제목":subject, "참여인원":0}, \
                          ignore_index=True)
  return request_people

In [137]:
url = "https://www1.president.go.kr/petitions?page="
df = pd.DataFrame(columns=['번호','분류','제목','참여인원'])

for a in range(10):
  url2 = url + str(a+1)
  html = get_html(url2)
  bs = get_div(html)
  df = html_to_df(bs, df)

df

TypeError: ignored

# 3. 임의의 청원 100개 가져오기
2018년 7월 17일 현재 국민청원의 글 번호는 1번부터 238663번까지 부여되어 있습니다. 하지만 중간중간 삭제된 청원이 있어서 실제 청원 수는 238663개보다 적습니다. 309510번 청원글 본문의 URL은 https://www1.president.go.kr/petitions/309510 입니다. URL에서 글번호만 바꾸면 해당 청원의 본문으로 이동할 수 있습니다. 이러한 URL 패턴의 특성을 활용하여 전체 청원 중 임의로 100개 청원의 제목과 본문을 추출하여 DataFrame에 담아주세요.

In [139]:
from random import *
url = "https://www1.president.go.kr/petitions/"
for a in range(100):
  rand = str(randint(1, 309510))
  html = get_html(url + rand)
  ## url이 잘못되어 가져오지 못할 경우 처리
  bs = get_div(html)
  df = html_to_df(bs, df)

TypeError: ignored